In [ ]:
#default_exp pycollections.list

# Generic List Operations

> This module houses generic operations on data held in lists.

In [ ]:
#hide
from nbdev.showdoc import *

To start, Python already has the [`reduce()`](https://docs.python.org/3/library/functools.html#functools.reduce) function in the `functools` library, but `reduce` is strictly less powerful than [`fold`](https://en.wikipedia.org/wiki/Fold_(higher-order_function)) insofar as it is designed to stay within the type boundary. That is, `reduce` takes a callable of type `(A, A) -> A`, while `fold` can break through the type boundary by taking a callable of `(B, A) -> B`. In some cases, Python's `reduce` seems to handle this just fine, but it isn't consistent. We can take care of this by forcing the developer to adhere to the contract in `fold`.

*Note that Python does not natively support tail call elimination.* Consequently, we must be mindful of blowing the stack. The imperative implementations of a normally recursive approach is motivated by this concern.

In [ ]:
#export
from typing import List, TypeVar, Callable, Dict
from dataclasses import dataclass, field


A = TypeVar("A")
B = TypeVar("B")


def foldl_list(a_vals: List[A], start: B, f: Callable[[B, A], B]) -> B:
    """
    Given a way to merge values of `A` with values of `B`, this function
    converts a list of values of type `A` into a single value of type `B`.
    Note that `B` can be the same as the input type (`List[A]`), but it
    need not be.
    """
    out: B = start
    for next_a in a_vals:
        out = f(out, next_a)
    return out

The `fold` pattern is quite generic. It can be used to simply add numbers up...

In [ ]:
xs: List[int] = list(range(5))
xs_sum: int = foldl_list(xs, 0, lambda b, a: b + a)
xs_sum

10

In [ ]:
assert foldl_list(xs, 0, lambda b, a: b + a) == 10 # This actually gets run when we run the test suite

... it can also be used to convert data between different type representations.

In [ ]:
@dataclass
class Foo:
    value: int
    even: bool = field(default_factory=bool)
        
    def __post_init__(self) -> None:
        self.even = self.value % 2 == 0
        

def allocate(out: Dict[str, List[Foo]], next_i: int) -> Dict[str, List[Foo]]:
    f: Foo = Foo(next_i)
    
    if f.even:
        out.update({"even": out["even"] + [f]})
    else:
        out.update({"odd": out["odd"] + [f]})
        
    return out

xs_to_foos: Dict[str, List[Foo]] = foldl_list(xs, {"even": [], "odd": []}, allocate)
xs_to_foos

{'even': [Foo(value=0, even=True),
  Foo(value=2, even=True),
  Foo(value=4, even=True)],
 'odd': [Foo(value=1, even=False), Foo(value=3, even=False)]}

As can be seen, the transformation is arbitrary. What makes this pattern powerful is that allows us to focus on a single transformation while `fold` takes care of traversal. 

The `l` is `foldl_list` indicates that we are [folding from the left](https://superruzafa.github.io/visual-scala-reference/foldLeft/), in the sense that we are updating our output value and carrying forward to the next value in the list. We can also [fold from the right](https://superruzafa.github.io/visual-scala-reference/foldRight/). Effectively, we wait until the last stack frame before resolving any of the calls. It is harder to avoid a recursive approach here, so we should use it with caution.

In [ ]:
#export
def foldr_list(a_vals: List[A], start: B, f: Callable[[A, B], B]) -> B:
    """
    Given a way to merge values of `A` with values of `B`, this function
    converts a list of values of type `A` into a single value of type `B`.
    Note that `B` can be the same as the input type (`List[A]`), but it
    need not be.
    
    This function is much like `foldl_list`, except that it resolves in the
    reverse order. Note also that `start` plays the role of the output variable
    after the first call.
    """
    if len(a_vals) == 0:
        return start
    else:
        next_a: A
        remaining_a_vals: List[A]
        next_a, *remaining_a_vals = a_vals
        return f(next_a, foldr_list(remaining_a_vals, start, f))

`foldr_list` is deployed in a very similar fashion to `foldl_list`.

In [ ]:
xs_sum2: int = foldr_list(xs, 0, lambda a, b: a + b)

xs_sum2

10

In [ ]:
assert foldr_list(xs, 0, lambda a, b: a + b) == 10

However, there is a difference that is more apparent when order is important. Suppose we wanted to use `fold` to implement a mapping operating. We can note that folding right will reverse the order unless we go out of our way to prevent it.

In [ ]:
map_left: Callable[[List[int], int], List[int]] = lambda b, a: b + [a]
map_right: Callable[[int, List[int]], List[int]] = lambda a, b: b + [a]
    
foldl_list(xs, [], map_left), foldr_list(xs, [], map_right)

([0, 1, 2, 3, 4], [4, 3, 2, 1, 0])

Given the concern about stack overflows, defaulting to `foldl_list` is a reasonable bias. The onus should be on the developer to demonstrate that `foldr_list` is more appropriate.